In [3]:
import json, os, sys, math, time, random, requests as req
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [4]:
import librosa 

In [5]:
# SPOTIPY_CLIENT_ID = '0387ebd733aa4a8e97ab976f87422cf1'
# SPOTIPY_CLIENT_SECRET = '0fbdb2b2366c4d44b01a7fb28c882de0'

SPOTIPY_CLIENT_ID = 'dfa8b23916c140f5b7c7572da25c4734'
SPOTIPY_CLIENT_SECRET = 'e65a6c357d80435fb629f41677abff3f'

# SPOTIPY_CLIENT_ID = '89adeaef38664c11988fd12defc1df71'
# SPOTIPY_CLIENT_SECRET = '9fd3b4fc50bb4002babf623768c18efd'

# SPOTIPY_CLIENT_ID = 'a455bd6d8a0d4be88191d23fd2ca86a0'
# SPOTIPY_CLIENT_SECRET = 'e40911234058421b884b0de3a819b198'

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [5]:
with open('msd-data/searchin.json', 'r') as f:
    searching = json.load(f)

In [34]:
AUDIO_FILE = 'funny.mp3'
def download_preview(song: dict):
    title, artist = song['title'], song['artist']
    try:
        search = sp.search(q=f"track:{title} artist:{artist}", offset=0, type='track', limit=1)
    except spotipy.SpotifyException as ex:
        if ex.code == -1 and ex.http_status == 400:
            return '', False
        else:
            raise

    tracks = search['tracks']['items']
    if len(tracks) == 0:
        return '', False
    
    url = tracks[0]['preview_url']
    
    return tracks[0]['id'], download_preview_url(url)

def download_preview_url(url: str):
    # print(url)
    if url == 'None' or url is None:
        return False

    session = req.Session()
    content = session.get(url).content

    with open(AUDIO_FILE, 'wb') as f:
        f.write(content)

    return True


In [7]:
searching_l = [(k, v) for k, v in searching.items()]
searching_l_copy = searching_l

In [8]:
searching_l_copy[5719]

('TRATMXM128F1457ACB',
 {'title': 'La Traviata - highlights (1990 Digital Remaster): Ah_ non più (Alfredo/Violetta)',
  'artist': 'Maria Callas/Giuseppe di Stefano/Ettore Bastianini/Silvana Zanolli/Luisa Mandelli/Giuseppe Zampieri/Arturo La Porta/Antonio Zerbini/Silvio Maionica/Coro e Orchestra del Teatro alla Scala_ Milano/Carlo Maria Giulini'})

In [ ]:
offset = 7099
for i, (id, value) in enumerate(searching_l_copy[offset:]):
    if i % 5 == 0:
        print(f'Analyzing {id} (#{i+offset}) | timestamp: {time.strftime("%I:%M:%S", time.localtime())}', end='\r')
    
    if not download_preview(value):
        continue
    
    y, sr = librosa.load(AUDIO_FILE, sr=44100)
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=2048, n_mfcc=15, hop_length=1024)
    mfcc = mfcc[:, :1200]
    listm = [[round(float(c), 3) for c in l] for l in mfcc]

    with open(f'msd-data/mfcc-features/{id}.json', 'w') as f:
        json.dump(listm, f)

    time.sleep(0.95)

In [5]:
to_explore = [('3lPr8ghNDBLc2uZovNyLs9', "https://p.scdn.co/mp3-preview/f4fe399267e6093182d576a6c84c0e081c81ff90?cid=9f313b6479844355af4354c1c85d1918")]
i = 0
while True:
    id, url = to_explore.pop(0)
    # if i % 5 == 0:
    print(f'Analyzing {id} (#{i}) | timestamp: {time.strftime("%I:%M:%S", time.localtime())}', end='\r')
        
    if not download_preview_url(url):
        continue

    y, sr = librosa.load(AUDIO_FILE, sr=44100)
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=2048, n_mfcc=15, hop_length=1024)
    mfcc = mfcc[:, :1200]
    listm = [[round(float(c), 3) for c in l] for l in mfcc]

    with open(f'msd-data/mfcc-features/{id}.json', 'w') as f:
        json.dump(listm, f)

    neighbors = sp.recommendations(seed_tracks=['33UFGRwKJzhi4H86Hd2Efs'])['tracks']
    # print(neighbors)
    if neighbors:
        to_explore.extend([(t['id'], t['preview_url']) for t in neighbors])

    time.sleep(2.5)

    i += 1

In [1]:
LAST_FM_API = '8844ca9d5f9c69c5ebd02e436be0f3e9'

In [68]:
def last_fm_get_simiar(artist: str, track: str) -> dict:
    params = {
        'api_key': LAST_FM_API,
        'method': 'track.getSimilar',
        'artist': artist,
        'track': track,
        'format': 'json',
        'autocorrect': 1
    }
    
    try:
        return req.get('http://ws.audioscrobbler.com/2.0', params=params).json()['similartracks']['track']
    except ConnectionResetError:
        return None
    except json.JSONDecodeError:
        return None

In [70]:
tracks = last_fm_get_simiar('Gunship', 'Blood for the blood god (feat. HEALTH)')
tracks

[{'name': 'Holographic Heart (feat. Britta Phillips)',
  'playcount': 14294,
  'match': 1.0,
  'url': 'https://www.last.fm/music/Gunship/_/Holographic+Heart+(feat.+Britta+Phillips)',
  'streamable': {'#text': '0', 'fulltrack': '0'},
  'artist': {'name': 'Gunship', 'url': 'https://www.last.fm/music/Gunship'},
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b

In [71]:
to_explore = [('Gunship', 'Blood for the blood god (feat. HEALTH)')]
explored = set([])
i = 0
while True:
    artist, track = to_explore.pop(0)

    # if (artist, track) in explored:
    #     continue

    explored.add((artist, track))

    id, success = download_preview({'title': track, 'artist': artist})
    
    if not success:
        continue
    
    # if i % 5 == 0:
    print(f'Analyzing {artist}|{track} (#{i}) | Track ID: {id} | timestamp: {time.strftime("%I:%M:%S", time.localtime())}', end='\r', flush=True)

    tracks = last_fm_get_simiar(artist, track)
    if tracks == None:
        print('Encountered an error oops')
        continue

    info_pairs = [pair for track in tracks if (pair := (track['artist']['name'], track['name'])) not in explored]
    to_explore.extend(info_pairs)

    if not os.path.exists(f'msd-data/mfcc-features/{id}.json'):

        y, sr = librosa.load(AUDIO_FILE, sr=44100)
        
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=2048, n_mfcc=15, hop_length=1024)
        mfcc = mfcc[:, :1200]
        listm = [[round(float(c), 3) for c in l] for l in mfcc]

        with open(f'msd-data/mfcc-features/{id}.json', 'w') as f:
            json.dump(listm, f)

        i += 1

    time.sleep(0.95)


In [48]:
[n for i in range(10) if (n := i) % 2 == 0]

[0, 2, 4, 6, 8]